In [5]:
import numpy as np
from itertools import product

select_ingre = []

#재료 선택
def __init__(self, ingre):
    self.ingre = ingre

def print_info(self):
    print("select ingredient 다 고르셨으면 'r' 입력\n a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : ", self.ingre)

def select():
    ingre = input("select ingredient 다 고르셨으면 'r' 입력\n a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : ")
    if(ingre < 'a' or ingre > 'q'):
        print("없는 품목입니다 다시 고르세요\n")
    else :
        select_ingre.append(ingre)
    while(1):
        ingre = input("select ingredient 다 고르셨으면 'r' 입력\n a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : ")
        if(ingre < 'a' or ingre > 'q' and ingre != 'r'):
            print("없는 품목입니다 다시 고르세요")
            continue
        elif(ingre == 'r'):
            break
        else:
            if ingre in select_ingre:
                print("중복입니다 다시 고르세요")
                continue
            else:
                select_ingre.append(ingre)
                continue
                
def run():
    select()
    
if __name__ == "__main__":
    run()

#선택한 재료 숫자 번호로 반환
for i in range (len(select_ingre)):
    select_ingre[i] = ord(select_ingre[i]) - ord('a')

#정렬
select_ingre.sort()
#print(select_ingre)

#재료이름 순서대로 array에 저장
ingredient = np.array(['사과', '배', '배추', '무', '양파', '상추', '오이', '호박', '쇠고기',
                       '돼지고기', '닭고기', '달걀(10개)', '달걀(30개)', '조기', '명태', '오징어', '고등어'])
select_ingredient = []
for i in select_ingre:
    select_ingredient.append(ingredient[i])
#print(select_ingredient)

#csv파일에서 시장이름정보 가져오기
all_market = np.genfromtxt('market_price_data_8.csv', delimiter = ',', usecols = 1, skip_header = 1, dtype = 'U')
market = np.zeros(49, dtype = 'U24')
for i in range(49):
    market[i] = all_market[(i + 1) * 17 - 17]

#csv파일에서 재료 가격정보 가져오기, 배열에 저장, 행 : 시장, 열 : 재료
price = np.zeros((49, 17), dtype = 'i')
for i in range(49):
    price[i] = np.genfromtxt('market_price_data_8.csv', delimiter = ',', usecols = 3, skip_header = i*17+1, max_rows = 17)

#품목별 가장 작은 가격만 남겨두고 모두 0으로 변환, 원래 0인 값은 비교에서 제외
tmp = np.full((17), 100000)
for j in range(17):
    for i in range(49):
        if(price[i][j] <= tmp[j] and price[i][j] > 0):
            tmp[j] = price[i][j]
        else:
            price[i][j] = 0

for j in range(17):
    for i in range(49):
        if(price[i][j] > tmp[j]):
            price[i][j] = 0

#선택한 재료의 col만 따로 분리
select_price = price[:, select_ingre]

#전치
select_price_T = select_price.T

#가격정보가 0이 아닌 index를 찾는 과정
price_index_a = np.where(select_price > 0)
price_index_b = np.zeros((len(price_index_a[0]), 2), dtype = 'i')

a = 0
for i in range(len(select_price_T)):
    for j in range(49):
        if(select_price_T[i][j] != 0):
            price_index_b[a] = (i, j)
            a += 1

#품목별 가장 싼 시장 출력
print("품목별 가장 싼 시장 목록")
for i in range(len(price_index_b)):
    print(select_ingredient[price_index_b[i][0]], "-", market[price_index_b[i][1]])

#시장들의 부분노드
select_tmp = []
mid_tmp = 1
middle = 1
for i in range(len(price_index_b) - 1):
    if(price_index_b[i][0] == price_index_b[i+1][0]):
        mid_tmp += 1
        if(i == (len(price_index_b) - 2)):
            select_tmp.append(mid_tmp)
    else:
        select_tmp.append(mid_tmp)
        mid_tmp = 1
        if(i == (len(price_index_b) - 2)):
            select_tmp.append(mid_tmp)

#print(price_index_b)

for i in range(len(price_index_b)):
    price_index_b[i][1] += 1

for i in range(len(select_tmp)):
    middle *= select_tmp[i]

mid = []
for i in range(price_index_b[len(price_index_b) - 1][0] + 1):
    mid.append([j[1] for j in price_index_b if j[0] == i])

mid_2 = list(product(*mid))

for i in range(middle):
    mid_2[i] = list(set(mid_2[i]))
    mid_2[i].append(0)
    mid_2[i].sort()
    
#mid_2 = list(map(int, mid_2))

#선택된 시장들의 집합
print(mid_2)

#거리정보 data csv파일 받아오기
dist = np.genfromtxt('distance_information.csv', delimiter = ',', usecols = range(1, 51) , skip_header = 1)
#print(dist)

# Process TSP
path_tmp = []
dis_tmp = 10000
a_tmp = 0
for a in range((len(mid_2))):
    distances = np.zeros((len(mid_2[a]), len(mid_2[a])))

    for i in range(len(mid_2[a])):
        for j in range(len(mid_2[a])):
            distances[i][j] = dist[int(mid_2[a][i])][int(mid_2[a][j])]

    #print(distances)
    def path_len(path):
        return round(sum(distances[i][j] for i, j in zip(path, path[1:])), 2)

    # Set of all nodes to visit
    to_visit = set(range(len(distances)))

    # Current state {(node, visited_nodes): shortest_path}
    state = {(i, frozenset([0, i])): [0, i] for i in range(1, len(distances[0]))}

    for _ in range(len(distances) - 2):
        next_state = {}
        for position, path in state.items():
            current_node, visited = position

            # Check all nodes that haven't been visited so far
            for node in to_visit - visited:
                new_path = path + [node]
                new_pos = (node, frozenset(new_path))

                # Update if (current node, visited) is not in next state or we found shorter path
                if new_pos not in next_state or path_len(new_path) < path_len(next_state[new_pos]):
                    next_state[new_pos] = new_path

        state = next_state

    # Find the shortest path from possible candidates
    shortest_tmp = min((path + [0] for path in state.values()), key=path_len)
    #print(len(shortest_tmp))
    #print ('path: {0}, length: {1}'.format(shortest_tmp, path_len(shortest_tmp)))
    if(path_len(shortest_tmp) >= dis_tmp):
        continue
    else:
        dis_tmp = path_len(shortest_tmp)
        path_tmp = shortest_tmp
        a_tmp = a
        
for i in range(len(path_tmp)):
    path_tmp[i] = mid_2[a_tmp][path_tmp[i]]

print('path: {0}, length: {1}'.format(path_tmp, dis_tmp))

#결과출력######################
print("최적의 경로 : 우리가게 -> ", end = '')
for i in range(1, len(path_tmp) - 1):
    print(market[path_tmp[i] - 1], " -> ", end = '')
print(" 우리가게")
print("거리 : ", dis_tmp, "km")

select ingredient 다 고르셨으면 'r' 입력
 a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : a
select ingredient 다 고르셨으면 'r' 입력
 a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : i
select ingredient 다 고르셨으면 'r' 입력
 a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : k
select ingredient 다 고르셨으면 'r' 입력
 a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : m
select ingredient 다 고르셨으면 'r' 입력
 a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : q
select ingredient 다 고르셨으면 'r' 입력
 a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : r
품목별 가장 싼 시장 목록
사과 - 대림중앙시장
사과 - 영등포전통시장
사과 - 신영시장
사과 - 망원시장
사과 - 마포농수산물시장
사과 - 남성시장
쇠고기 - 남성시장
닭고기 - 청량리종합시장
닭고기 - 경동시장
닭고기 - 동원시장
닭고기 - 남성시장
달걀(30개) - 영천시장
고등어 - 장위골목시장
고등어 - 원당